In [19]:
from dataclasses import dataclass, field  # Import field
import csv
import os
import random
import pandas as pd

parsed_before_folder = os.path.join("Parsed Data", "Before")
parsed_after_folder = os.path.join("Parsed Data", "After")

before_files = [f for f in os.listdir(parsed_before_folder) if f.endswith('.csv')] #Getting all before files
after_files = [f for f in os.listdir(parsed_after_folder) if f.endswith('.csv')] #Getting all after files

In [ ]:
@dataclass
class Influencer:
    name: str
    affiliation: str
    no_followers: int = 0
    like: int = 0
    retweet: int = 0
    reply: int = 0
    views: int = 0


before_data = []
after_data = []

before_corpus = []
after_corpus = []

#Loading File paths
supplementary_folder = "Supplementary Materials"
influencers_path = os.path.join(supplementary_folder, "Followers List & Categories - Accounts Kept.csv")

def getting_values(files: list, path: str, after: bool):
    for f in files:
        with open(f"{path}/{f}", 'r') as csvfile:
            reader = csv.reader(csvfile)
            next(reader) #skip header
            
            for line in reader:
                if not line:  # Skip empty lines
                    continue
                if line[0] == "|RUN STATISTICS|": # End of file, move on to next file
                    break

                name = line[0].strip() if line else ""

                tweet = line[2].strip()
                date = line[1]
                like = int(line[3].strip())
                retweet = int(line[4].strip())
                reply = int(line[5].strip())
                view = int(line[6].strip())


                if after and tweet not in after_corpus:
                    after_corpus.append(tweet)
                    before_data.append(Influencer(name, "", 0, like, retweet, reply, view))
                else:
                    if tweet not in before_corpus:
                        before_corpus.append(tweet)
                        after_data.append(Influencer(name, "", 0, like, retweet, reply, view))

getting_values(before_files, parsed_before_folder, False)
getting_values(after_files, parsed_after_folder, True)

with open(influencers_path, 'r') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)
    for line in reader:
        followers = line[2].strip()
        affiliation = line[1].strip()
        name = line[0].strip()
        
        for i in before_data:
            if i.name == name[1:]:
                i.affiliation = affiliation
                i.no_followers = followers
        
        for i in after_data:
            if i.name == name[1:]:
                i.affiliation = affiliation
                i.no_followers = followers


valid = []
nb = []
na = []

for i in before_data:
    if i.name not in nb:
        nb.append(i.name)
for i in after_data:
    if i.name not in na:
        na.append(i.name)
for name in nb:
    if name in na:
        valid.append(name)

    
with open("Before Graphing.csv", 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Name", "Affiliation", "Followers", "Likes", "Retweets", "Replies", "Views"])
    for i in before_data:
        if i.affiliation and i.name in valid:
            writer.writerow([i.name, i.affiliation, i.no_followers, i.like, i.reply, i.retweet, i.views])

with open("After Graphing.csv", 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Name", "Affiliation", "Followers", "Likes", "Retweets", "Replies", "Views"])
    for i in after_data:
        if i.affiliation and i.name in valid:
            writer.writerow([i.name, i.affiliation, i.no_followers, i.like, i.reply, i.retweet, i.views])